In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import get_file
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses

In [9]:
base_url = 'https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip'
zip_file = get_file("Flowers_data.zip", base_url, extract=True, cache_dir='data/')
zip_file

311443456/311442766 [==============================] - 21s 0us/step


'/tmp/.keras/datasets/Flowers_data.zip'

In [76]:
base_path='/tmp/.keras/datasets/flower_data'

train_data = ImageDataGenerator(rescale=1./255, horizontal_flip=True, rotation_range=20, validation_split=0.2)
train_dataset = train_data.flow_from_directory(
    base_path+"/train", target_size=(256, 256), class_mode='sparse'
)

len(train_dataset)

Found 6552 images belonging to 102 classes.


205

In [58]:
test_data = ImageDataGenerator(rescale=1./255)
test_dataset = test_data.flow_from_directory(
    base_path+"/valid", target_size=(256, 256), class_mode="sparse")

Found 818 images belonging to 102 classes.


In [0]:
img, labels = train_dataset.next()

In [71]:
img.shape

(32, 256, 256, 3)

In [0]:
base_model = applications.DenseNet121(
    input_shape=(256, 256, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

In [0]:
model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(102, activation='softmax')
])

In [65]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 8, 8, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1024)              0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 102)               52326     
Total params: 7,614,630
Trainable params: 577,126
Non-trainable params: 7,037,504
______________________________________

In [66]:
len(model.trainable_variables)

4

In [0]:
model.compile(
    optimizers.Adam(learning_rate=0.003),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [72]:
history = model.fit(train_dataset, workers=4, epochs=1)

Epoch 1/10
205/205 [==============================] - 118s 577ms/step - loss: 1.9905 - acc: 0.4637
Epoch 2/10
 21/205 [==>...........................] - ETA: 1:55 - loss: 1.6616 - acc: 0.5283

KeyboardInterrupt: ignored

In [73]:
model.evaluate(test_dataset, workers=4)

26/26 [==============================] - 15s 587ms/step - loss: 3.2698 - acc: 0.2604


[3.2698115293796244, 0.2603912]

# Fine Tune

In [0]:
base_model.trainable = True

In [96]:
len(base_model.trainable_variables)

279

In [0]:
for layer in base_model.layers[:200]:
  layer.trainable = False

In [0]:
model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(102, activation='softmax')
])

In [99]:
len(model.trainable_variables)

198

In [0]:
model.compile(
    optimizers.Adam(learning_rate=0.003),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [102]:
history = model.fit(train_dataset, workers=4, epochs=2)

Epoch 1/2
40/40 [==============================] - 56s 1s/step - loss: 4.7549 - acc: 0.0638
Epoch 2/2
40/40 [==============================] - 26s 647ms/step - loss: 4.0154 - acc: 0.1237


In [103]:
model.evaluate(test_dataset, workers=4)

26/26 [==============================] - 8s 318ms/step - loss: 42.1103 - acc: 0.0636


[42.11026147695688, 0.06356968]